## In progress:
Developing semi-supervised toy CNN model

Weekend goals:
- automate data augmentation
- add unsupervised learning

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import skimage as sk
from skimage import transform
import random
from PIL import Image
import os
import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# # compress images

# """ compresses all .jpg files in the directory and subdirectory to have a certain basewidth"""
# def compress_im(root,basewidth):
#     print(f"compress imgs in {root}")
    
#     for ob in os.listdir(root):
#         file_path = os.path.join(root,ob)
#         if os.path.isdir(file_path):
#             compress_im(file_path,basewidth)
#         elif ob[-3:] == "jpg":
#             #print(file_path)
#             #print(os.path.getsize(file_path))
#             img = Image.open(file_path)
#             img.show()
#             comp_frac = basewidth/img.size[0]
#             assert (comp_frac <=1.)
#             hsize = int(img.size[1] * comp_frac)
#             img = img.resize((basewidth, hsize), Image.ANTIALIAS)
#             img.show()
#             img.save(file_path)
#             #print(os.path.getsize(file_path))
#             #assert 0 ==1
            

# compress_im("Labeled/Flooded",1000)
# compress_im("Labeled/Non_Flooded",1000)
# compress_im("Unlabeled",1000)

compress imgs in Labeled/Flooded
compress imgs in Labeled/Non_Flooded
compress imgs in Unlabeled


In [13]:
# cd drive/MyDrive/Colab\ Notebooks/AM\ 231\ Toy\ Model/
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
cd drive/MyDrive

/content/drive/MyDrive


In [17]:
def rotate_img(image):
    random_degree = random.uniform(-25, 25) #25% from left or right
    return sk.transform.rotate(image, random_degree)

def noise_img(image):
    return sk.util.random_noise(image)

In [18]:
# EC changed filepaths

flooded_img = []
nonflooded_img = []
unlabeled_img = []

for file in os.listdir('Labeled/Flooded/'):
    image = Image.open(os.path.join('Labeled/Flooded/', file))
    image = np.array(image.resize((1000,750)))
    flooded_img.append(rotate_img(image))

for file in os.listdir('Labeled/Non_Flooded/'):
    image = Image.open(os.path.join('Labeled/Non_Flooded/', file))
    image = np.array(image.resize((1000,750)))
    nonflooded_img.append(rotate_img(image))
    
for file in os.listdir('Unlabeled/'):
    image = Image.open(os.path.join('Unlabeled/', file))
    image = np.array(image.resize((1000,750)))
    unlabeled_img.append(rotate_img(image))
    unlabeled_img.append(rotate_img(image))

In [19]:
print("Flooded Image Shape: {}".format(flooded_img[0].shape))
print("Non_Flooded Image Shape: {}".format(nonflooded_img[0].shape))
print("Unlabeled Image Shape: {}".format(unlabeled_img[0].shape))

Flooded Image Shape: (750, 1000, 3)
Non_Flooded Image Shape: (750, 1000, 3)
Unlabeled Image Shape: (750, 1000, 3)


In [34]:
unlabeled_img = unlabeled_img / 255.
data_img = np.vstack((np.array(flooded_img), np.array(nonflooded_img))) / 255.
data_img.shape

(20, 750, 1000, 3)

In [35]:
unlabeled_img = np.array(unlabeled_img)
unlabeled_img.shape

(28, 750, 1000, 3)

In [36]:
train_idx = np.array([np.arange(7),np.arange(10,17)]).flatten()
test_labeled_idx = np.array([np.arange(7,9),np.arange(17,19)]).flatten()
test_unlabeled_idx = np.array([9,19]).flatten()
print("Training Index: {}".format(train_idx))
print("Testing Labeled Index: {}".format(test_labeled_idx))
print("Testing Unlabeled Index: {}".format(test_unlabeled_idx))

Training Index: [ 0  1  2  3  4  5  6 10 11 12 13 14 15 16]
Testing Labeled Index: [ 7  8 17 18]
Testing Unlabeled Index: [ 9 19]


### Example Tensorflow CNN Code

Source: [Link1](https://medium.com/data-science-group-iitr/building-a-convolutional-neural-network-in-python-with-tensorflow-d251c3ca8117), [Link2] (https://www.datacamp.com/community/tutorials/cnn-tensorflow-python)

##### Helper functions for building CNN

In [23]:
# convolutional layer
def conv_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[-1]]))

    out = tf.nn.conv2d(input=x, filters=weights, strides=[1,1,1,1], padding='SAME')
    out += bias
    return out

# pooling layer
def max_pool(x, k=2):

    out = tf.nn.max_pool(value=x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    return out

# fully connected layer
def fully_connected_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[1]]))

    out = tf.matmul(a=x, b=weights)
    out += bias
    return out

# flatten layer
def flatten_layer(x):
    
    size = x.get_shape()[1:4].num_elements()
    out = tf.reshape(x, [-1,size])
    return out, size

# relu
relu = lambda x: tf.nn.relu(features=x)

# softmax
softmax = lambda x: tf.nn.softmax(logits=x)

##### Define toy CNN

In [24]:
#shape = [filter_size, filter_size, num_input_channels, num_filters]

# define CNN
def toy_model(x):

    # three convolutional layers with max pool
    shape0 = [5, 5, 3, 3]
    conv0 = conv_layer(x, shape0)
    conv0 = max_pool(conv0, k=2)

    shape1 = [5, 5, 3, 1]
    conv1 = conv_layer(conv0, shape1)
    conv1 = max_pool(conv1, k=2)

    shape2 = [5, 5, 1, 1]
    conv2 = conv_layer(conv1, shape2)
    conv2 = max_pool(conv2, k=2)

    # flatten output and put through a fully connected layer
    flat1, size1 = flatten_layer(conv2)
    fc1 = fully_connected_layer(flat1, [size1, 64])
    fc1 = relu(fc1)

    fc2 = fully_connected_layer(fc1, [64, 1])
    out = softmax(fc2)

    return out

##### Build model with placeholder tensors

In [29]:
def generate_guess_label(pred_u_raw, k):
    # guess label = average prediction over k augmentations of same image
    # num_images = pred_u_raw.shape[0].value / k # Throws error of NoneType and int since pred_u_raw.shape[0].value is None
    
    try:
      num_images = int(pred_u_raw.shape[0].value / k)

      idx = 0
      temp_labels = []
      for i in range(num_images):
          temp_labels.append(tf.reduce_mean(pred_u_raw[idx:idx+k]))
          idx += k

      # repeat label for each augmentation
      guess_labels = tf.repeat(tf.stack(temp_labels), k)

      # reshape and remove gradient tracking
      guess_labels = tf.reshape(guess_labels, (-1,1))
      guess_labels = tf.stop_gradient(guess_labels)

      return guess_labels

    except TypeError:
      
      return pred_u_raw

    

In [37]:
# define inputs
hdim = data_img[0].shape[0]
vdim = data_img[0].shape[1]
x = tf.placeholder(tf.float32, [None, hdim, vdim, 3]) # labeled images (augmented)
u = tf.placeholder(tf.float32, [None, hdim, vdim, 3]) # unlabeled images (augmented)
y = tf.placeholder(tf.float32, [None, 1]) # labels
y_train_true = np.array([np.ones(7),np.zeros(7)]).reshape(-1,1)
y_test_true = np.array([np.ones(2),np.zeros(2)]).reshape(-1,1)
k = 2 # augment images k times

# Google paper section 3.5 says 100 is a good place to start for w_unlabeled
# Google paper also suggests ramping up value to 100 over first 16,000 epochs
w_unlabeled = 100. 

# run model with placeholder tensors
pred_x = toy_model(x)
pred_u_raw = toy_model(u)

# calculate guess labels for unlabeled images 
pred_u = generate_guess_label(pred_u_raw, k)

# sharpen guess labels here? is this necessary for only 2 classes?

# define loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=pred_x, labels=y)
labeled_loss = tf.reduce_mean(cross_entropy)
unlabeled_loss = tf.nn.l2_loss(pred_u - pred_u_raw)
cost = labeled_loss + w_unlabeled*unlabeled_loss

# define accuracy
pred_correct = tf.equal(tf.argmax(pred_x, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

# initialize variables
init = tf.global_variables_initializer()
training_iters = 3
batch_size = len(train_idx)

##### Train model

In [38]:
# train model
with tf.Session() as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    
    for i in range(training_iters):
   
        # Run optimization 
        # Calculate batch loss and accuracy
        for batch in range(len(train_idx)//batch_size):
            batch_x = data_img[train_idx,:,:,:][batch*batch_size:min((batch+1)*batch_size, len(train_idx))]
            batch_u = unlabeled_img[:,:,:,:][batch*k*batch_size:min((batch+1)*k*batch_size, unlabeled_img.shape[0])]
            batch_y = y_train_true[batch*batch_size:min((batch+1)*batch_size, len(y_train_true))]   

            feed_dict={x: batch_x, u: batch_u, y: batch_y}
            opt = sess.run(optimizer, feed_dict=feed_dict)
            loss, acc = sess.run([cost, accuracy], feed_dict=feed_dict)

        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))

        # Calculate accuracy for all test images
        test_acc,valid_loss = sess.run([accuracy,cost],
                                feed_dict={x: data_img[test_labeled_idx,:,:,:], u: data_img[test_unlabeled_idx,:,:,:], y : y_test_true})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 0.000000, Training Accuracy= 1.00000
Testing Accuracy: 1.00000
Iter 1, Loss= 0.000000, Training Accuracy= 1.00000
Testing Accuracy: 1.00000
Iter 2, Loss= 0.000000, Training Accuracy= 1.00000
Testing Accuracy: 1.00000
